In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/delhi-neighborhood-data/restaurant_dataSet.csv
/kaggle/input/delhi-neighborhood-data/delhi_dataSet.csv
/kaggle/input/neighborhoods-of-gurgaon-with-latlong/df_latlong.csv


In [2]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import requests
from pandas.io.json import json_normalize

# Data Collection
We are collecting data for __Delhi__, using __Foursquare API__ and Kaggle Datasets 

In [3]:
Neighbourhood_df= pd.read_csv('../input/delhi-neighborhood-data/delhi_dataSet.csv')
Neighbourhood_df.head(100)

,Unnamed: 0,Borough,Neighborhood,latitude,longitude
0,0,North West Delhi,Adarsh Nagar,28.614192,77.071541
1,1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,2,North West Delhi,Azadpur,28.707657,77.175547
3,3,North West Delhi,Bawana,28.799660,77.032885
4,4,North West Delhi,Begum Pur,NaN,NaN
5,5,North West Delhi,Dhaka,39.031714,-90.261223
6,6,North West Delhi,Jahangirpuri,28.725972,77.162658
7,7,North West Delhi,Karala,28.735140,77.032511
8,8,North West Delhi,Keshav Puram,28.688926,77.161683
9,9,North West Delhi,Kingsway Camp,28.614262,77.201555


In [4]:
CLIENT_ID = '1YZJO3UY3Y1IXCYCE3WLWZWKEOU3GEK2MBP15OSG0QH5GKR4'
CLIENT_SECRET = 'VGX2KLUEJPDOLLWC043AZKN0FRF2RVMN20M5OIG5UYGSGYZF'
VERSION = '20180605'

print('MY CLIENT_ID: ' + CLIENT_ID)
print('MY CLIENT_SECRET:' + CLIENT_SECRET)

MY CLIENT_ID: 1YZJO3UY3Y1IXCYCE3WLWZWKEOU3GEK2MBP15OSG0QH5GKR4
MY CLIENT_SECRET:VGX2KLUEJPDOLLWC043AZKN0FRF2RVMN20M5OIG5UYGSGYZF


In [5]:
Neighbourhood_df.dropna( inplace=True)
Neighbourhood_df.head()

,Unnamed: 0,Borough,Neighborhood,latitude,longitude
0,0,North West Delhi,Adarsh Nagar,28.614192,77.071541
1,1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,2,North West Delhi,Azadpur,28.707657,77.175547
3,3,North West Delhi,Bawana,28.799660,77.032885
5,5,North West Delhi,Dhaka,39.031714,-90.261223


In [6]:
latitude = 28.7041
longitude =  77.1025
print('The geograpical coordinate of Delhi are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Delhi are 28.7041, 77.1025.


In [7]:
import folium as fo

In [8]:
delhi_map = fo.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, label in zip(Neighbourhood_df['latitude'], Neighbourhood_df['longitude'], Neighbourhood_df['Neighborhood']):
    label = fo.Popup(label, parse_html=True)
    fo.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#a45b2c',
        fill_opacity=0.7,
        parse_html=False).add_to(delhi_map)  
    
delhi_map

In [15]:
CLIENT_ID = '1YZJO3UY3Y1IXCYCE3WLWZWKEOU3GEK2MBP15OSG0QH5GKR4'
CLIENT_SECRET = 'VGX2KLUEJPDOLLWC043AZKN0FRF2RVMN20M5OIG5UYGSGYZF'
VERSION = '20180605'

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
      
        results = requests.get(url).json()["response"]['groups'][0]['items']
      
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues) 

In [19]:
delhi_venues= getNearbyVenues(names=Neighbourhood_df['Neighborhood'],latitudes=Neighbourhood_df['latitude'],longitudes=Neighbourhood_df['longitude'])

ConnectionError: HTTPSConnectionPool(host='api.foursquare.com', port=443): Max retries exceeded with url: /v2/venues/explore?&client_id=1YZJO3UY3Y1IXCYCE3WLWZWKEOU3GEK2MBP15OSG0QH5GKR4&client_secret=VGX2KLUEJPDOLLWC043AZKN0FRF2RVMN20M5OIG5UYGSGYZF&v=20180605&ll=28.614192499999998,77.0715411848447&radius=500&limit=100 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fb7d56cc650>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))